<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/%5B20250531%5D%20Trust%20FL%20MLP-LSTM_logged_extended%20on%20WSN-BFSF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***20250531 Trust FL MLP-LSTM_logged_extended on WSN-BFSF***

In [1]:
# trust_federated_mlp_lstm_logged_extended.py

import pandas as pd
import numpy as np
import time
import psutil
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, InputLayer
from tensorflow.keras.regularizers import l2
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.utils import to_categorical
from imblearn.over_sampling import SMOTE
from scipy.spatial.distance import cosine

In [2]:
# Load dataset
dataset = pd.read_csv("dataset.csv")  # WSN-BFSF 4-class dataset
dataset = dataset.dropna()  # Remove missing values

In [3]:
dataset

,Event,Time,S_Node,Node_id,Rest_Energy,Trace_Level,Mac_Type_Pckt,Source_IP_Port,Des_IP_Port,Packet_Size,TTL,Hop_Count,Broadcast_ID,Dest_Node_Num,Dest_Seq_Num,Src_Node_ID,Src_Seq_Num,Class
0,1,0.100000,79,79,600.000000,5,0,79.255,1.255,48,30,1,1,100,0,79,4,0
1,2,0.100963,78,78,599.979723,5,800,79.255,1.255,48,30,1,1,100,0,79,4,0
2,2,0.100963,76,76,599.979722,5,800,79.255,1.255,48,30,1,1,100,0,79,4,0
3,2,0.100964,75,75,599.979722,5,800,79.255,1.255,48,30,1,1,100,0,79,4,0
4,2,0.100964,118,118,599.979722,5,800,79.255,1.255,48,30,1,1,100,0,79,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312101,4,656.526536,140,140,465.332078,5,800,140.255,1.255,48,30,1,31,100,16,140,68,1
312102,4,657.209643,140,140,465.191806,5,800,140.255,1.255,48,30,1,33,100,16,140,72,1
312103,4,657.891729,140,140,465.051737,5,800,140.255,1.255,48,30,1,35,100,16,140,76,1
312104,4,658.574656,140,140,464.911501,5,800,140.255,1.255,48,30,1,37,100,16,140,80,1


In [4]:
# Encode non-numeric columns (if any)
for column in dataset.columns:
    if dataset[column].dtype == 'object':  # Identify categorical columns
        dataset[column] = LabelEncoder().fit_transform(dataset[column])

# Separate features (X) and target (y)
X = dataset.drop(['Class'], axis=1)  # Assuming 'Class' is the target column
y = dataset['Class']

print("Original Class Distribution:", np.bincount(y))

Original Class Distribution: [262851  11766   7645  29844]


In [5]:
from sklearn.preprocessing import LabelEncoder, StandardScaler # Import StandardScaler here
# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

from imblearn.over_sampling import SMOTE

# Apply SMOTE to balance the data
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

# Verify the balanced dataset distribution
print("Balanced Class Distribution:", np.bincount(y))

Balanced Class Distribution: [262851 262851 262851 262851]


In [6]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101, stratify=y)

In [7]:
pip install matplotlib

In [8]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, TimeDistributed
from tensorflow.keras.regularizers import l2
import numpy as np

In [9]:
from sklearn.metrics import roc_curve, auc
from itertools import cycle

y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

input_shape = (X_train.shape[1], X_train.shape[2])
num_classes = y_train_cat.shape[1]

y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

input_shape = (X_train.shape[1], X_train.shape[2])
num_classes = y_train_cat.shape[1]

# -------------------------
# Utility Functions
# -------------------------
def build_model(input_shape, num_classes):
    model = Sequential([
        InputLayer(input_shape=input_shape),
        LSTM(64, activation='tanh', kernel_regularizer=l2(0.01)),
        Dropout(0.3),
        Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(0.3),
        Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def compute_cosine_similarity(update1, update2):
    flat1 = np.concatenate([w.flatten() for w in update1])
    flat2 = np.concatenate([w.flatten() for w in update2])
    return 1 - cosine(flat1, flat2)

def compute_stability(update, history):
    if len(history) < 2:
        return 1.0
    sims = [compute_cosine_similarity(update, past) for past in history[-3:]]
    return np.mean(sims)

def normalize(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val + 1e-8)

def compute_trust_scores(client_updates, global_weights, val_losses, update_history, alpha=(0.4, 0.4, 0.2)):
    scores = {}
    losses = list(val_losses.values())
    min_loss, max_loss = min(losses), max(losses)
    for cid in client_updates:
        sim = compute_cosine_similarity(client_updates[cid], global_weights)
        loss = val_losses[cid]
        stability = compute_stability(client_updates[cid], update_history.get(cid, []))
        trust = alpha[0] * sim + alpha[1] * (1 - normalize(loss, min_loss, max_loss)) + alpha[2] * stability
        scores[cid] = trust
    return scores

def trust_weighted_aggregation(client_weights, trust_scores):
    total_trust = sum(trust_scores.values())
    return [sum(trust_scores[cid] * np.array(client_weights[cid][i]) for cid in client_weights) / total_trust
            for i in range(len(next(iter(client_weights.values()))))]

# -------------------------
# Federated Training with Profiling
# -------------------------
clients = 5
rounds = 30
client_X = np.array_split(X_train, clients)
client_y = np.array_split(y_train_cat, clients)

global_model = build_model(input_shape, num_classes)
global_weights = global_model.get_weights()
update_history = {i: [] for i in range(clients)}
trust_log, comm_log, round_log = [], [], []

# Model size estimation
model_params = global_model.count_params()
param_size_MB = sum(w.nbytes for w in global_weights) / (1024 ** 2)

for r in range(rounds):
    start_time = time.time()
    round_comm = 0
    local_weights = {}
    val_losses = {}

    for cid in range(clients):
        local_model = build_model(input_shape, num_classes)
        local_model.set_weights(global_weights)
        local_model.fit(client_X[cid], client_y[cid], epochs=1, batch_size=32, verbose=0)
        weights = local_model.get_weights()
        loss = local_model.evaluate(X_test, y_test_cat, verbose=0)[0]
        local_weights[cid] = weights
        val_losses[cid] = loss
        round_comm += sum(w.nbytes for w in weights)
        update_history[cid].append(weights)

    trust_scores = compute_trust_scores(local_weights, global_weights, val_losses, update_history)
    for cid in trust_scores:
        trust_log.append({'round': r+1, 'client_id': cid, 'trust_score': trust_scores[cid]})

    global_weights = trust_weighted_aggregation(local_weights, trust_scores)
    global_model.set_weights(global_weights)

    pred = global_model.predict(X_test)
    pred_classes = np.argmax(pred, axis=1)
    true_classes = np.argmax(y_test_cat, axis=1)
    acc = accuracy_score(true_classes, pred_classes)
    prec = precision_score(true_classes, pred_classes, average='weighted')
    rec = recall_score(true_classes, pred_classes, average='weighted')
    f1 = f1_score(true_classes, pred_classes, average='weighted')
    cm = confusion_matrix(true_classes, pred_classes)
    duration = (time.time() - start_time) * 1000  # ms

    comm_log.append({'round': r+1, 'comm_cost_MB': round_comm / (1024 ** 2)})
    round_log.append({
        'round': r+1,
        'accuracy': acc,
        'precision': prec,
        'recall': rec,
        'f1_score': f1,
        'round_time_ms': duration
    })

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


6572/6572 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


6572/6572 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


...

6572/6572 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


6572/6572 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


6572/6572 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step


In [10]:
# Create the directory if it doesn't exist
if not os.path.exists("/mnt/data"):
    os.makedirs("/mnt/data")

# Save Logs
pd.DataFrame(trust_log).to_csv("/mnt/data/trust_scores_log.csv", index=False)
pd.DataFrame(comm_log).to_csv("/mnt/data/communication_cost_log.csv", index=False)
pd.DataFrame(round_log).to_csv("/mnt/data/resource_and_performance_log.csv", index=False)

# Model summary
model_profile = {
    "Parameters (MB)": param_size_MB,
    "FLOPs (estimated)": model_params * 2,
    "Peak Memory (MB)": psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2),
    "Total Rounds": rounds,
    "Clients": clients
}
pd.DataFrame([model_profile]).to_csv("/mnt/data/model_resource_profile.csv", index=False)

In [11]:
# prompt: Download all Logs .csv files aboved

!zip logs.zip /mnt/data/*.csv
from google.colab import files
files.download('logs.zip')


  adding: mnt/data/communication_cost_log.csv (deflated 84%)
  adding: mnt/data/model_resource_profile.csv (deflated 4%)
  adding: mnt/data/resource_and_performance_log.csv (deflated 57%)
  adding: mnt/data/trust_scores_log.csv (deflated 57%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>